# I will do this in a jupyter notebook. First I create a way to read the TofTracer2 files. 

In [ ]:
using HDF5

# Start playing around with my own results file in which I want to store my plots

### First I create a dummy plot and save it as such.

In [ ]:
using Plots

xarr = 1:0.1:100
yarr = sin.(xarr)
plt = plot(xarr, yarr, label="Sine Wave", title="Dummy Plot", xlabel="X-axis", ylabel="Y-axis")
savefig(plt, joinpath("..","plots","dummy_plot.png"))


 ### Then I declare a function that (over)writes the plot data to an h5 file in the folder "files" as well as a function that clears the whole file.

In [ ]:
function save_plot_data_to_hdf5(;id = "__currentPlot", title = "Dummy Title", xdata::Vector = [1,2,3], ydata::Vector = sin.(xdata), file = joinpath("..", "files", "my_plot_data.hdf5"))    
    my_results = h5open(file, "cw")

    "Plots" in keys(my_results) || create_group(my_results, "Plots")

    id ∉ keys(my_results["Plots"]) || delete_object(my_results, "Plots/"*id) # Delete existing plot data if present

    create_group(my_results["Plots"], id) # Create a new group for the plot
    
    my_results["Plots"][id]["x"] = xdata;
    my_results["Plots"][id]["y"] = ydata;
    my_results["Plots"][id]["title"] = title;

    close(my_results)
end;

function clear_file(;file = joinpath("..", "files", "my_plot_data.hdf5"))    
    
    my_results = h5open(file, "cw")
    
    "Plots" ∉ keys(my_results) || delete_object(my_results, "Plots")
    display(my_results)
    close(my_results)
end;

### Some Tests

In [ ]:
ids = [1,2,1]
for i in ids
    save_plot_data_to_hdf5(id="Testplot$(i)", title="Dummy Plot $(i)", xdata=collect(xarr), ydata=yarr)
end

file = joinpath("..", "files", "my_plot_data.hdf5")
my_results = h5open(file, "cw")
display(my_results)
close(my_results)

In [ ]:
clear_file()

# Now I am ready to look at the TofTracer2 examples

### !!! I copied the files into the project folder to make it easier to use.

In [ ]:
using HDF5

raw_data_file = joinpath("..", "files", "2016-10-02-19h15m05.h5")
results_file = joinpath("..", "files", "_result.hdf5")

### First lets look at the raw data

In [ ]:
raw_data = h5open(raw_data_file, "r")
display(keys(raw_data))

In [ ]:
display(raw_data["AcquisitionLog"])
t_window = read(raw_data["AcquisitionLog/Log"]) # Time window information
display(t_window)

### Above we have some general information about when this spectrum was taken. Below we have the raw data which we can plot, including the summed data as well as the individual spectra. I am not quite shure, where the 4th dimension comes from, maybe 5s resolution chunked to 10s?

In [ ]:
mass_axis = read(raw_data["FullSpectra/MassAxis"]) # Mass axis 368000x1 array
sum_spectrum = read(raw_data["FullSpectra/SumSpectrum"]) # Sum spectrum 368000x1 array
tof_data = read(raw_data["FullSpectra/TofData"]); # ToF data 368000x1 x2x60 array

### I am also interested in some timing data

In [ ]:
timing_data = read(raw_data["TimingData/BufTimes"]) # Timing data 2x60 Matrix

# Lets Plot

In [ ]:
using Plots
using Dates


function plot_spectrum(
    mass_axis::Vector,
    spectrum::Vector;
    title::String="Spectrum Plot",
    xlabel::String="m/z",
    ylabel::String="Counts",
    yscale::Symbol=:log10,
    ylims::Tuple=(max(minimum(spectrum),1), maximum(spectrum)*1.1),
    time_label::String="",
    pngname::String="__current_spectrum_plot",
    savepath::String=joinpath("..","plots",pngname*".png"),
    show::Bool=true,
    saveplot::Bool=true,
    plt = plot()
)

    if yscale == :log10
        spectrum = map(x -> max(x, 1), spectrum) # Avoid log10(<=0)
    end

    plt = plot!(mass_axis, spectrum, xlabel=xlabel, ylabel=ylabel, title=title, label=title*"\n"*time_label, yscale = yscale, ylims = ylims)

    if show
        display(plt)
    end
    if saveplot
        savefig(plt, savepath)
    end

    return plt
end;


In [ ]:


t1 = DateTime(t_window[1][2], DateFormat("dd/mm/yyyy HH:MM:SS"))
t2 = DateTime(t_window[end][2], DateFormat("dd/mm/yyyy HH:MM:SS"))

time_label(t1::DateTime, t2::DateTime) = string(Dates.format(t1, "yyyy-mm-dd"))*" - " *string(Dates.format(t1, "HH:MM:SS"))*" to " *string(Dates.format(t2, "HH:MM:SS"))

plt = plot_spectrum(
    mass_axis,
    sum_spectrum;
    title="Sum Spectrum from Raw Data",
    time_label=time_label(t1,t2));

save_plot_data_to_hdf5(id="SumSpectrum_Raw", title="Sum Spectrum from Raw Data", xdata=mass_axis, ydata=sum_spectrum);

### Now I plot the first minute

In [ ]:
selection = 1:(12+1) # 12 * 5 s = 60 seconds = first minute - but we want end time not start of last 5s interval

timing_data = reshape(timing_data, :) # flatten to 1D array
tof_data = reshape(tof_data, size(tof_data,1), size(tof_data,2), :) # ensure tof_data is 3D array
tof_data_first_minute = reshape(sum(tof_data[:,:,selection], dims = 3), size(tof_data,1)); # select first minute data

print(size(tof_data_first_minute))


In [ ]:

t1_firt_minute = t1 + Millisecond(round(Int, timing_data[selection[1]] * 1000)) # adjust time_label
t2_first_minute = t1 + Millisecond(round(Int, timing_data[selection[end]] * 1000))

plt = plot_spectrum(
    mass_axis,
    tof_data_first_minute;
    title="Spectrum first minute from Raw Data",
    pngname="Spectrum_1st_min_Raw",
    time_label=time_label(t1_firt_minute,t2_first_minute));

save_plot_data_to_hdf5(id="Spectrum_1st_min_Raw", title="Sum Spectrum from Raw Data", xdata=mass_axis, ydata=sum_spectrum);

In [ ]:
close(raw_data)

# And now the Results file

In [ ]:
results_data = h5open(results_file, "r");
for el in keys(results_data)
    println("Element: ", el, " \t Size: ", size(read(results_data[el])))
end

### This helps a lot! I am just a bit confused about the dimension with 5 entries.

In [ ]:
mass_axis = read(results_data["MassAxis"]); #368000 Vector

spec = read(results_data["AvgSpectrum"]); #368000 Vector
baseline = read(results_data["AvgBaseline"]); #368000 Vector
sum_spec_max = read(results_data["SumSpecMax"]); #368000 Vector
sum_spec_min = read(results_data["SumSpecMin"]); #368000 Vector


plt = plot() # Now I plot all four lines in one diagram

for (label, curve) in zip(
    ["Average Spectrum", "Average Baseline", "Sum Spectrum Max", "Sum Spectrum Min"],
    [spec, baseline, sum_spec_max, sum_spec_min]
)
    plt = plot_spectrum(
        mass_axis,
        curve;
        title=label,
        pngname=replace(label, " "=>"_")*"_from_Results",
        show=false,
        saveplot=false,
        plt=plt
        );
    
    save_plot_data_to_hdf5(id=replace(label, " "=>"_")*"_Results", title=label*" from Results Data", xdata=mass_axis, ydata=curve);
end 

display(plt)
savefig(plt, joinpath("..","plots","Results_Data_Spectra.png"));

### This does not reveal much, but it is a nice spectrum :)

In [ ]:
plot(mass_axis, sum_spec_max-sum_spec_min)

In [ ]:
close(results_data)

# Finally lets look at what we saved. 
### Of course one could add more details like axis names and descriptions etc. 

In [ ]:
my_results = h5open(file, "cw")

In [ ]:
close(my_results)